# Python 3.10 Setup Instructions

## Problem
Your original `requirements.txt` was created on a **Linux** machine with Python 3.10. This machine is **Windows** with Python 3.13. There are two issues:
1. Python 3.13 is too new for some NVIDIA CUDA packages
2. Some NVIDIA packages (like `nvidia-cufile-cu12`, `nvidia-nccl-cu12`, `nvidia-nvshmem-cu12`) are **Linux-only** and not available on Windows

## Solution: Install Python 3.10 and Use Windows-Compatible Requirements

### Step 1: Install Python 3.10
Run in PowerShell:
```powershell
winget install Python.Python.3.10
```

### Step 2: Verify Installation
After installation, close and reopen your terminal, then run:
```powershell
py -3.10 --version
```

### Step 3: Create New Virtual Environment with Python 3.10
Navigate to your project directory and create a new virtual environment:
```powershell
cd C:\Users\as2491\git\image_deblurring
py -3.10 -m venv venv310
```

### Step 4: Activate the New Virtual Environment
```powershell
.\venv310\Scripts\Activate.ps1
```

### Step 5: Install Windows-Compatible Requirements
I've created `requirements-windows.txt` with Linux-specific packages removed:
```powershell
pip install --upgrade pip
pip install -r requirements-windows.txt
```

**Note:** PyTorch will automatically install the Windows-compatible CUDA libraries it needs.

### Step 6: Select Kernel in VS Code
1. Click the kernel selector in the top-right of this notebook
2. Select the Python 3.10 interpreter from `venv310`

---

In [15]:
def solution(A):
    sorted_a = sorted(A)
    smallest = 1
    for num in sorted_a:
        if num == smallest:
            smallest +=1
        elif num > smallest:
            break
    return smallest

A = [1, 3, 6, 4, 1, 2]
print(solution(A))
    

5


In [2]:
print(6+3)

9


In [3]:
import sys
print(f"python ver: {sys.version}")

python ver: 3.10.12 (main, Aug 15 2025, 14:32:43) [GCC 11.4.0]


In [3]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("CUDA device count:", torch.cuda.device_count())
if torch.cuda.is_available():
    print("Current device:", torch.cuda.current_device())
    print("Device name:", torch.cuda.get_device_name(torch.cuda.current_device()))
else:
    print("No GPU detected.")

CUDA available: False
CUDA device count: 0
No GPU detected.


/home/andrei/git/image_deblurring/virtual_env/lib/python3.10/site-packages/torch/cuda/__init__.py:827: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [4]:
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
import os
from PIL import Image
import numpy as np

# get images eaxct string location
sharp_dir = "/home/andrei/git/image_deblurring/data/sharp"
input_dir = "/home/andrei/git/image_deblurring/data/input"
output_dir = "/home/andrei/git/image_deblurring/data/output"

sharp_filenames = [f for f in os.listdir(sharp_dir) if f.endswith(".png")]
sharp_img_path = os.path.join(sharp_dir, sharp_filenames[0])

blurry_filenames = [f for f in os.listdir(input_dir) if f.endswith(".png")]
blurry_img_path = os.path.join(input_dir, blurry_filenames[0])

output_filenames = [f for f in os.listdir(output_dir) if f.endswith(".png")]
output_img_path = os.path.join(output_dir, output_filenames[0])

# check how to use PIL to make them into an Image object - then check if the Image object has values between 0 and 1.

pil_sharp = Image.open(sharp_img_path).convert("RGB")
pil_blurry = Image.open(blurry_img_path).convert("RGB")
pil_output = Image.open(output_img_path).convert("RGB")

np_sharp = np.array(pil_sharp)
np_blurry = np.array(pil_blurry)
np_output = np.array(pil_output)


# Baseline: Blurry vs Sharp (no deblurring)
psnr_baseline = psnr(np_sharp, np_blurry, data_range=255)
ssim_baseline = ssim(np_sharp, np_blurry, channel_axis=-1, data_range=255)

psnr_output = psnr(np_sharp, np_output, data_range=255)
ssim_output = ssim(np_sharp, np_output, channel_axis=-1, data_range=255)

print(f"Baseline (no deblurring): PSNR={psnr_baseline:.2f}, SSIM={ssim_baseline:.3f}")
print("----")
print(f"output: PSNR={psnr_output:.2f}, SSIM={ssim_output:.3f}")

Baseline (no deblurring): PSNR=24.05, SSIM=0.695
----
output: PSNR=27.76, SSIM=0.818


Baseline (no deblurring): PSNR=20.58, SSIM=0.527

output: PSNR=24.26, SSIM=0.717